In [87]:
import MySQLdb
import simplejson as json
import requests
from bs4 import BeautifulSoup
import re

db = MySQLdb.connect(host="localhost", port=3306,
                     user="root", 
                      passwd="mysql", 
                      db="civic")

url = "http://myneta.info/delhi2015/index.php?action=show_winners&sort=default"
headers = {'Accept': 'text/html'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content)
tables = soup.findAll("table")
urls = []
politicianNames = []
for idx, table in enumerate(tables):
    if(idx == 2):
        allrows = table.find_all('tr')
        del allrows[0]
        del allrows[0]
        for row in allrows:
            cells = row.find_all("td")
            del cells[0]
            for colIndex, cell in enumerate(cells):
                if(colIndex == 0):
                    for anchorIndex, a in enumerate(cell.findAll('a')):
                        if(anchorIndex == 1):
                            politicianNames.append(a.contents[0])
                            if a.has_key('href'):
                                arr = a['href'].split('=')
                                urls.append("http://myneta.info/delhi2015/candidate.php?candidate_id=" + arr[len(arr)-1])
                                break
for ix, u in enumerate(urls):
    politician = [None] * 4
    print ix+1, u
    r = requests.get(u, headers=headers)
    soup = BeautifulSoup(r.content)
    mydivs = soup.findAll("div", { "class" : "grid_3 alpha" })
    mytables = soup.findAll("table", { "class" : "table1" })
    for dx, tab in enumerate(mytables):
        if(not(dx == 0)):
            break
        allrow = tab.find_all('tr')
        cells = allrow[0].find_all("td")
        for idx3, cell in enumerate(cells):
            if(not(idx3 == len(cells)-1)):
                continue
            if(len(cell.contents)>0):
                del cell.contents[0]
                politician[0] = cell.text 
            
    for idx, div in enumerate(mydivs):
        if(idx==0):
            innerdivs = div.findAll("div", { "class" : "grid_2 alpha" })
            for idx2, div2 in enumerate(innerdivs):
                if(idx2==3):
                    politician[1] = div2.contents[2].strip()
                if(idx2==4):
                    if(len(div2.contents) > 1):
                        politician[2] = div2.contents[1]
                if(idx2==5):
                    politician[3] = div2.contents[1].strip()
    cur = db.cursor() 
    query = 'update c_politician cp set address = %s, email = %s, contact = %s, ipc = %s where name = %s'
    print politician[1], politician[2], politician[3], politician[0], politicianNames[ix]
    cur.execute(query, (politician[1], politician[2], politician[3], politician[0], politicianNames[ix]))
    db.commit()
    cur.close ()
db.close ()

1 http://myneta.info/delhi2015/candidate.php?candidate_id=210
Resident -13/C Gali no 36, Mahendra Park Delhi 110033 pksaapadarshnagar@gmail.com 9811139625  1 charges related to Causing death by negligence (IPC Section-304A)1 charges related to Negligent conduct with respect to machinery (IPC Section-287)  Pawan Kumar Sharma
2 http://myneta.info/delhi2015/candidate.php?candidate_id=216
K-579, Dakshin Puri, Dr Ambedkar Nagar N.D-110062 heyjay7@gmail.com 9953607523  Ajay Dutt
3 http://myneta.info/delhi2015/candidate.php?candidate_id=177
377 Gali no.2A, Matamandir marg, Maujpur gopalrai1975@gmail.com 9871215875  1 charges related to  criminal intimidation (IPC Section-506)1 charges related to Voluntarily causing hurt to deter public servant from his duty (IPC Section-332)2 charges related to  rioting (IPC Section-147)2 charges related to Rioting, armed with deadly weapon (IPC Section-148)1 charges related to Mischief causing damage to the amount of fifty rupees (IPC Section-427)1 charges r